
Installing Spark on google collab



In [309]:
#Install JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [310]:
# Donwload Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [311]:
# Unzip spark folder
!tar xf /content/spark-3.1.2-bin-hadoop2.7.tgz

In [312]:
!pip install -q findspark

In [313]:
# Create env variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [314]:
# Import findspark, that helps find the Spark on the system and import as a regular lib
import findspark
findspark.init()

Donwload files traffic violations zipped folder

In [430]:
import re
# Let's first download the data from the government site, this is one of my updates on the tutor´s code
# I download the code automatically from the site
import requests
import zipfile

def download_and_extract_file(url, destination):
    response = requests.get(url)
    if response.status_code == 200:
        with open(destination, 'wb') as file:
            file.write(response.content)
        print("File downloaded successfully.")

        # Extract the file
        with zipfile.ZipFile(destination, 'r') as zip_ref:
            zip_ref.extractall(os.path.dirname(destination))
        print("File extracted successfully.")

        # Delete the .zip file
        os.remove(destination)
        print(".zip file deleted.")
    else:
        print("Failed to download the file.")

In [316]:
# Download 2022 traffic violation data
file_url = 'https://drive.google.com/u/0/uc?id=1-oY2e1W4cjmbZCgdcGnmPCIod4MFCSpg&export=download&confirm=t&uuid=c1ebf929-8b62-4489-a46d-5284482747fb&at=AKKF8vwfciU3oT68KlBtF4jj14Wl:1688604985049'
file_destination = '/content/data/infracoes_2022.zip'

download_and_extract_file(file_url, file_destination)

File downloaded successfully.
File extracted successfully.
.zip file deleted.


In [317]:
# Download 2023 traffic violation data
file_url = 'https://drive.google.com/u/0/uc?id=106pwbYv84gCtLhhcbi0B_4qyUQxfH2n3&export=download&confirm=t&uuid=3967f839-2af2-437b-8f60-bb84c592767d&at=AKKF8vxhyEJdMfwlL_zzAfR9M7ZO:1688605862920'
file_destination = '/content/data/infracoes_2023.zip'

download_and_extract_file(file_url, file_destination)

File downloaded successfully.
File extracted successfully.
.zip file deleted.


Appeding each file content on a conselidated dataframe

In [431]:
from pyspark.sql import SparkSession
import os

def create_dataframe_with_file_content(directory_path):
    # List the files in the directory
    files = os.listdir(directory_path)

    # Create an empty list to store the DataFrames
    dataframes = []
    print("Processing files:")
    # Loop to consolidate the files
    for counter, file_name in enumerate(files):
        print(counter, file_name)

        # Read the file into a DataFrame
        file_path = os.path.join(directory_path, file_name)
        base_data = spark.read.format("csv").option("header", "true").option("sep", ";").option("encoding", "latin1").load(file_path)

        # Append the DataFrame to the list
        dataframes.append(base_data)

    # Union all the DataFrames in the list
    consolidated_data = dataframes[0]
    for i in range(1, len(dataframes)):
        consolidated_data = consolidated_data.union(dataframes[i])

    # Get the shape of the consolidated DataFrame
    print("Files content were consolidated.")
    print((consolidated_data.count(), len(consolidated_data.columns)))
    print("")
    return consolidated_data

In [432]:
# Create a SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('TVextraction')\
        .getOrCreate()

# Creating Traffic violation 2022 dataframe
TV22 = create_dataframe_with_file_content('/content/data/ajustados_2022')

Processing files:
0 infraçoes2022_01.csv
1 infraçoes2022_10.csv
2 infraçoes2022_02.csv
3 infraçoes2022_11.csv
4 infraçoes2022_09.csv
5 infraçoes2022_04.csv
6 infraçoes2022_08.csv
7 infraçoes2022_12.csv
8 infraçoes2022_03.csv
9 infraçoes2022_07.csv
10 infraçoes2022_05.csv
11 infraçoes2022_06.csv
Files content were consolidated.
(4135523, 25)



In [320]:
# Checking columns types
TV22.printSchema()

root
 |-- Número do Auto: string (nullable = true)
 |-- Data da Infração (DD/MM/AAAA): string (nullable = true)
 |-- Indicador de Abordagem: string (nullable = true)
 |-- Assinatura do Auto: string (nullable = true)
 |-- Sentido Trafego: string (nullable = true)
 |-- UF Infração: string (nullable = true)
 |-- BR Infração: string (nullable = true)
 |-- Km Infração: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Indicador Veiculo Estrangeiro: string (nullable = true)
 |-- UF Placa: string (nullable = true)
 |-- Descrição Especie Veículo: string (nullable = true)
 |-- Descrição Marca Veículo: string (nullable = true)
 |-- Descrição Tipo Veículo: string (nullable = true)
 |-- Descrição Modelo Veiculo: string (nullable = true)
 |-- Código da Infração: string (nullable = true)
 |-- Descrição Abreviada Infração: string (nullable = true)
 |-- Enquadramento da Infração: string (nullable = true)
 |-- Início Vigência da Infração: string (nullable = true)
 |-- Fim Vigência 

In [321]:
TV22.show(10)

+---------------+-----------------------------+----------------------+------------------+---------------+-----------+-----------+-----------+---------+-----------------------------+--------+-------------------------+-----------------------+----------------------+------------------------+------------------+----------------------------+-------------------------+---------------------------+---------------------+----------------+-------------+-------------------+------------------+-------------+
| Número do Auto|Data da Infração (DD/MM/AAAA)|Indicador de Abordagem|Assinatura do Auto|Sentido Trafego|UF Infração|BR Infração|Km Infração|Município|Indicador Veiculo Estrangeiro|UF Placa|Descrição Especie Veículo|Descrição Marca Veículo|Descrição Tipo Veículo|Descrição Modelo Veiculo|Código da Infração|Descrição Abreviada Infração|Enquadramento da Infração|Início Vigência da Infração|Fim Vigência Infração|Medição Infração|Hora Infração|Medição Considerada|Excesso Verificado|Qtd Infrações|
+-----

In [322]:
# Creating Traffic violation 2022 dataframe
TV23 = create_dataframe_with_file_content('/content/data/ajustados_2023')

Processing files:
0 infraçoes2023_05.csv
1 infraçoes2023_03.csv
2 infraçoes2023_04.csv
3 infraçoes2023_01.csv
4 infraçoes2023_02.csv
Files content were consolidated.
(2105730, 25)



In [323]:
TV23.printSchema()

root
 |-- Número do Auto: string (nullable = true)
 |-- Data da Infração (DD/MM/AAAA): string (nullable = true)
 |-- Indicador de Abordagem: string (nullable = true)
 |-- Assinatura do Auto: string (nullable = true)
 |-- Sentido Trafego: string (nullable = true)
 |-- UF Infração: string (nullable = true)
 |-- BR Infração: string (nullable = true)
 |-- Km Infração: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Indicador Veiculo Estrangeiro: string (nullable = true)
 |-- UF Placa: string (nullable = true)
 |-- Descrição Especie Veículo: string (nullable = true)
 |-- Descrição Marca Veículo: string (nullable = true)
 |-- Descrição Tipo Veículo: string (nullable = true)
 |-- Descrição Modelo Veiculo: string (nullable = true)
 |-- Código da Infração: string (nullable = true)
 |-- Descrição Abreviada Infração: string (nullable = true)
 |-- Enquadramento da Infração: string (nullable = true)
 |-- Início Vigência da Infração: string (nullable = true)
 |-- Fim Vigência 

In [324]:
TV23.show(10)

+---------------+-----------------------------+----------------------+------------------+---------------+-----------+-----------+-----------+---------+-----------------------------+--------+-------------------------+-----------------------+----------------------+------------------------+------------------+----------------------------+-------------------------+---------------------------+---------------------+----------------+-------------+-------------------+------------------+-------------+
| Número do Auto|Data da Infração (DD/MM/AAAA)|Indicador de Abordagem|Assinatura do Auto|Sentido Trafego|UF Infração|BR Infração|Km Infração|Município|Indicador Veiculo Estrangeiro|UF Placa|Descrição Especie Veículo|Descrição Marca Veículo|Descrição Tipo Veículo|Descrição Modelo Veiculo|Código da Infração|Descrição Abreviada Infração|Enquadramento da Infração|Início Vigência da Infração|Fim Vigência Infração|Medição Infração|Hora Infração|Medição Considerada|Excesso Verificado|Qtd Infrações|
+-----

Saving dataframe as parquet

In [325]:
from pyspark.sql.functions import col

def df_to_parquet(consolidated_data, file_path):
  # Convert columns to string type
  for col_name in consolidated_data.columns:
      # Remove invalid characters from the column name
      new_col_name = col_name.replace(" ", "_").replace(",", "").replace(";", "").replace("{", "").replace("}", "").replace("(", "").replace(")", "").replace("\n", "").replace("\t", "").replace("=", "")
      # Rename the column
      consolidated_data = consolidated_data.withColumnRenamed(col_name, new_col_name).withColumn(new_col_name, col(new_col_name).cast("string"))

  consolidated_data.write.parquet(file_path, compression="gzip")
  print("Parquet file was created and saved on " + file_path)

In [ ]:
# Save the TV23 dataFrame as a Parquet file
df_to_parquet(TV22,"/content/traffic_violations_2022.parquet")

In [51]:
# Save the TV23 dataFrame as a Parquet file
df_to_parquet(TV23,"/content/traffic_violations_2023.parquet")

Parquet file was created and saved on /content/traffic_violations_2023.parquet


In [327]:
# Close the SparkSession of Extraction
# I don't want to use the same spark context on the next steps
spark.stop()

End of the extraction and load section

Exploratory Data Analysis

Import parquet files

In [433]:
# Import Traffic violation parquet files as a spark dataframe

# Create a SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('TVanalysis')\
        .getOrCreate()

# Read 2022 traffic violations
TV22 = spark.read.parquet("/content/traffic_violations_2022.parquet")
# TV22 Shape
print("2022 Traffic violations Dataframe shape:")
print((TV22.count(), len(TV22.columns)))

2022 Traffic violations Dataframe shape:
(4135523, 25)


In [434]:
# Read 2023 traffic violations
TV23 = spark.read.parquet("/content/traffic_violations_2023.parquet")
#TV23 Shape
print("2023 Traffic violations Dataframe shape:")
print((TV23.count(), len(TV23.columns)))

2023 Traffic violations Dataframe shape:
(2105730, 25)


In [435]:
print("Dataframe type:")
type(TV22)

Dataframe type:


pyspark.sql.dataframe.DataFrame

In [436]:
print("2022 Traffic violations Dataframe schema:")
TV22.printSchema()

2022 Traffic violations Dataframe schema:
root
 |-- Número_do_Auto: string (nullable = true)
 |-- Data_da_Infração_DD/MM/AAAA: string (nullable = true)
 |-- Indicador_de_Abordagem: string (nullable = true)
 |-- Assinatura_do_Auto: string (nullable = true)
 |-- Sentido_Trafego: string (nullable = true)
 |-- UF_Infração: string (nullable = true)
 |-- BR_Infração: string (nullable = true)
 |-- Km_Infração: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Indicador_Veiculo_Estrangeiro: string (nullable = true)
 |-- UF_Placa: string (nullable = true)
 |-- Descrição_Especie_Veículo: string (nullable = true)
 |-- Descrição_Marca_Veículo: string (nullable = true)
 |-- Descrição_Tipo_Veículo: string (nullable = true)
 |-- Descrição_Modelo_Veiculo: string (nullable = true)
 |-- Código_da_Infração: string (nullable = true)
 |-- Descrição_Abreviada_Infração: string (nullable = true)
 |-- Enquadramento_da_Infração: string (nullable = true)
 |-- Início_Vigência_da_Infração: str

In [437]:
from pyspark.sql.functions import col, sum as spark_sum
# Count the number of null values in each column
null_counts = TV22.select([spark_sum(col(c).isNull().cast("integer")).alias(c) for c in TV22.columns])
# Display the null counts
print("Count the number of null values in each column:")
null_counts.show()

Count the number of null values in each column:
+--------------+---------------------------+----------------------+------------------+---------------+-----------+-----------+-----------+---------+-----------------------------+--------+-------------------------+-----------------------+----------------------+------------------------+------------------+----------------------------+-------------------------+---------------------------+---------------------+----------------+-------------+-------------------+------------------+-------------+
|Número_do_Auto|Data_da_Infração_DD/MM/AAAA|Indicador_de_Abordagem|Assinatura_do_Auto|Sentido_Trafego|UF_Infração|BR_Infração|Km_Infração|Município|Indicador_Veiculo_Estrangeiro|UF_Placa|Descrição_Especie_Veículo|Descrição_Marca_Veículo|Descrição_Tipo_Veículo|Descrição_Modelo_Veiculo|Código_da_Infração|Descrição_Abreviada_Infração|Enquadramento_da_Infração|Início_Vigência_da_Infração|Fim_Vigência_Infração|Medição_Infração|Hora_Infração|Medição_Considerad

In [438]:
from pyspark.sql.functions import countDistinct
# Calculate the number of unique values in each column
nunique_counts = TV22.agg(*[countDistinct(col_name).alias(col_name) for col_name in TV22.columns])
# Display the number of unique values
print("Calculate the number of unique values in each column:")
nunique_counts.show()

Calculate the number of unique values in each column:
+--------------+---------------------------+----------------------+------------------+---------------+-----------+-----------+-----------+---------+-----------------------------+--------+-------------------------+-----------------------+----------------------+------------------------+------------------+----------------------------+-------------------------+---------------------------+---------------------+----------------+-------------+-------------------+------------------+-------------+
|Número_do_Auto|Data_da_Infração_DD/MM/AAAA|Indicador_de_Abordagem|Assinatura_do_Auto|Sentido_Trafego|UF_Infração|BR_Infração|Km_Infração|Município|Indicador_Veiculo_Estrangeiro|UF_Placa|Descrição_Especie_Veículo|Descrição_Marca_Veículo|Descrição_Tipo_Veículo|Descrição_Modelo_Veiculo|Código_da_Infração|Descrição_Abreviada_Infração|Enquadramento_da_Infração|Início_Vigência_da_Infração|Fim_Vigência_Infração|Medição_Infração|Hora_Infração|Medição_Cons

In [439]:
# Get the total number of rows
total_multa = int(TV22.count())
# Print the total number of multas
print(f'Total of 2022 Traffic Violations: {total_multa}')

Total of 2022 Traffic Violations: 4135523


In [440]:
from pyspark.sql.functions import col, to_date
# Convert the column to date type
TV22 = TV22.withColumn('Data_da_Infração_DD/MM/AAAA', to_date(col('Data_da_Infração_DD/MM/AAAA'), 'yyyy-MM-dd'))
# Print the updated data type of the column
TV22.printSchema()

root
 |-- Número_do_Auto: string (nullable = true)
 |-- Data_da_Infração_DD/MM/AAAA: date (nullable = true)
 |-- Indicador_de_Abordagem: string (nullable = true)
 |-- Assinatura_do_Auto: string (nullable = true)
 |-- Sentido_Trafego: string (nullable = true)
 |-- UF_Infração: string (nullable = true)
 |-- BR_Infração: string (nullable = true)
 |-- Km_Infração: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Indicador_Veiculo_Estrangeiro: string (nullable = true)
 |-- UF_Placa: string (nullable = true)
 |-- Descrição_Especie_Veículo: string (nullable = true)
 |-- Descrição_Marca_Veículo: string (nullable = true)
 |-- Descrição_Tipo_Veículo: string (nullable = true)
 |-- Descrição_Modelo_Veiculo: string (nullable = true)
 |-- Código_da_Infração: string (nullable = true)
 |-- Descrição_Abreviada_Infração: string (nullable = true)
 |-- Enquadramento_da_Infração: string (nullable = true)
 |-- Início_Vigência_da_Infração: string (nullable = true)
 |-- Fim_Vigência_Infr

In [441]:
from pyspark.sql.functions import expr
from pyspark.sql.window import Window

# Creating a temporary view called TV22
TV22.createOrReplaceTempView("TV22")

# Creating a query, to get the contect from the temp view
query = "SELECT\
            `Data_da_Infração_DD/MM/AAAA` AS `Date`, \
            COUNT(*) AS `Violations_Count`, \
            MONTH(`Data_da_Infração_DD/MM/AAAA`) AS `Month`, \
            DAY(`Data_da_Infração_DD/MM/AAAA`) AS `Day`\
        FROM \
            TV22 \
        GROUP BY \
            `Data_da_Infração_DD/MM/AAAA` \
        ORDER BY \
            `Data_da_Infração_DD/MM/AAAA`"

# run the query and storying in another variable called flights10
violations_per_day = spark.sql(query)

# Get the moving avarage
window_spec = Window.orderBy("Date").rowsBetween(-6, 0)
violations_per_day = violations_per_day.withColumn("moving_avarage", expr("avg(Violations_Count)").over(window_spec))

# see result
violations_per_day.show(10)

+----------+----------------+-----+---+------------------+
|      Date|Violations_Count|Month|Day|    moving_avarage|
+----------+----------------+-----+---+------------------+
|2022-01-01|            8533|    1|  1|            8533.0|
|2022-01-02|           13146|    1|  2|           10839.5|
|2022-01-03|           12012|    1|  3|11230.333333333334|
|2022-01-04|           11248|    1|  4|          11234.75|
|2022-01-05|           11159|    1|  5|           11219.6|
|2022-01-06|           10167|    1|  6|11044.166666666666|
|2022-01-07|           11150|    1|  7|11059.285714285714|
|2022-01-08|            8321|    1|  8|           11029.0|
|2022-01-09|            8788|    1|  9| 10406.42857142857|
|2022-01-10|            8520|    1| 10|  9907.57142857143|
+----------+----------------+-----+---+------------------+
only showing top 10 rows



In [442]:
violations_per_day.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Violations_Count: long (nullable = false)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- moving_avarage: double (nullable = true)



In [443]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [444]:
!pip install plotly

In [445]:
# Import pandas lib
import pandas as pd

# Converting Spark Dataframe to Pandas Dataframe
df_violations_per_day = violations_per_day.toPandas()

In [446]:
px.line(
    df_violations_per_day,
    x='Date',
    y='Violations_Count'
)

In [447]:
# Boxplot
px.box(
    df_violations_per_day,
    x='Month',
    y='Violations_Count',
    color='Month',
    title='Tickets per Month Distribution'
)

Month with more outliers: October and June

*   List item
*   List item


Month with the best median: November

In [448]:
# Grid System

# Grid
Grid = make_subplots( rows=1, cols= 2 )

# Graph 1
Grid.add_trace(
    go.Scatter(
        x=df_violations_per_day['Date'],
        y=df_violations_per_day['Violations_Count'],
        mode='lines',
        name='Violations Count'
    ),
    row=1, col=1
)

# Graph 2
Grid.add_trace(
    go.Scatter(
        x=df_violations_per_day['Date'],
        y=df_violations_per_day['moving_avarage'],
        mode='lines',
        name='Moving Avarage'
    ),
    row=1, col=2
)

# Layout Adjust
Grid.update_layout(

    # title
    title='Traffic Violation Analyses per Day - Grid System',

    # legend
    showlegend=True,

    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

Grid

In [449]:
# Grid
Grid = make_subplots( rows=1, cols= 1 )

# Graph 1
Grid.add_trace(
    go.Scatter(
        x=df_violations_per_day['Date'],
        y=df_violations_per_day['Violations_Count'],
        mode='lines',
        name='Violations Count'
    ),
    row=1, col=1
)

# Graph 2
Grid.add_trace(
    go.Scatter(
        x=df_violations_per_day['Date'],
        y=df_violations_per_day['moving_avarage'],
        mode='lines',
        name='Moving Avarage'
    ),
    row=1, col=1
)

# Layout Adjust
Grid.update_layout(

    # Title
    title='Traffic Violation Analyses per Day',

    # Legend
    showlegend=True,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

Grid

Lets have a closer look on the Brazilian States

In [450]:
from pyspark.sql.functions import count, sum
from pyspark.sql.window import Window

# Calculate the sum of infractions by state
violations_per_state = TV22.groupBy('UF_Infração').agg(count('*').alias('Violations_Count'))

# Calculate the percentage of infractions for each state
state_percent = (TV22.groupBy("UF_Infração").agg((count("*") / TV22.count()).alias("%violations_Count"))).orderBy(col("%violations_Count").desc())

# Calculate the percentage of infractions for each state (cummulative)
windowSpec = Window.orderBy(col("%violations_Count").desc())
cum_stata_percent= (
    state_percent
    .withColumn("cumulative_sum", sum("%violations_Count").over(windowSpec))
)

# Join the calculated frequencies with the violations_per_state DataFrame
violations_per_state = violations_per_state.join(cum_stata_percent, "UF_Infração", "left").orderBy(col("%violations_Count").desc())

# Print the DataFrame
violations_per_state.show(27)

+-----------+----------------+--------------------+-------------------+
|UF_Infração|Violations_Count|   %violations_Count|     cumulative_sum|
+-----------+----------------+--------------------+-------------------+
|         MG|          448672| 0.10849220279998442|0.10849220279998442|
|         RJ|          418981| 0.10131269974801252|0.20980490254799694|
|         BA|          417092| 0.10085592559876949|0.31066082814676643|
|         SP|          378074| 0.09142108507194859|  0.402081913218715|
|         RS|          252059| 0.06094972751934882| 0.4630316407380638|
|         MS|          248186| 0.06001320751933915| 0.5230448482574029|
|         PR|          237359| 0.05739515896780165| 0.5804400072252046|
|         GO|          230375| 0.05570637619474006| 0.6361463834199447|
|         SC|          178040| 0.04305138672907877| 0.6791977701490235|
|         MT|          172734| 0.04176835674713936| 0.7209661268961629|
|         ES|          150991| 0.03651073878684752| 0.7574768656

In [451]:
# Import pandas lib
import pandas as pd

# Converting Spark Dataframe to Pandas Dataframe
df_violations_per_state = violations_per_state.toPandas()

In [452]:
df_violations_per_state.head()

,UF_Infração,Violations_Count,%violations_Count,cumulative_sum
0,MG,448672,0.108492,0.108492
1,RJ,418981,0.101313,0.209805
2,BA,417092,0.100856,0.310661
3,SP,378074,0.091421,0.402082
4,RS,252059,0.060950,0.463032


In [453]:
# Gráfico de funil
px.funnel(
    df_violations_per_state[ df_violations_per_state['cumulative_sum'] < 0.5 ],
    y='UF_Infração',
    x='Violations_Count',
    title='State Concentration of 50% Traffic Infractions'
)

In [454]:
px.bar(
    df_violations_per_state,
    x='UF_Infração',
    y='cumulative_sum',
    title='Accumulated Infraction by state'
)

In [455]:
TV22.printSchema()

root
 |-- Número_do_Auto: string (nullable = true)
 |-- Data_da_Infração_DD/MM/AAAA: date (nullable = true)
 |-- Indicador_de_Abordagem: string (nullable = true)
 |-- Assinatura_do_Auto: string (nullable = true)
 |-- Sentido_Trafego: string (nullable = true)
 |-- UF_Infração: string (nullable = true)
 |-- BR_Infração: string (nullable = true)
 |-- Km_Infração: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Indicador_Veiculo_Estrangeiro: string (nullable = true)
 |-- UF_Placa: string (nullable = true)
 |-- Descrição_Especie_Veículo: string (nullable = true)
 |-- Descrição_Marca_Veículo: string (nullable = true)
 |-- Descrição_Tipo_Veículo: string (nullable = true)
 |-- Descrição_Modelo_Veiculo: string (nullable = true)
 |-- Código_da_Infração: string (nullable = true)
 |-- Descrição_Abreviada_Infração: string (nullable = true)
 |-- Enquadramento_da_Infração: string (nullable = true)
 |-- Início_Vigência_da_Infração: string (nullable = true)
 |-- Fim_Vigência_Infr

In [456]:
# Creating a query, to get the contect from the temp view
query = "SELECT * \
        FROM \
            TV22"

query = "SELECT\
            `UF_Infração` AS `UF`, \
            COUNT(*) AS `Violations_Count`, \
            MONTH(`Data_da_Infração_DD/MM/AAAA`) AS `Month` \
        FROM \
            TV22 \
        GROUP BY \
            `UF_Infração`, `Month` \
        ORDER BY \
            `Month`"

# run the query and storying in another variable called flights10
violations_month_state = spark.sql(query)

# see result
violations_month_state.show()

+---+----------------+-----+
| UF|Violations_Count|Month|
+---+----------------+-----+
| RS|           21648|    1|
| RN|            5390|    1|
| RR|             931|    1|
| SP|           35375|    1|
| PR|           28441|    1|
| AC|            1935|    1|
| DF|            3789|    1|
| MS|           19957|    1|
| ES|            9814|    1|
| AP|            1065|    1|
| PB|            8921|    1|
| BA|           29613|    1|
| MT|           15282|    1|
| TO|            3320|    1|
| SE|            3347|    1|
| CE|           11819|    1|
| MA|            6152|    1|
| MG|           25886|    1|
| RJ|           47616|    1|
| PI|            6897|    1|
+---+----------------+-----+
only showing top 20 rows



In [457]:
# Perform the pivot operation
pivot_violations_month_state = violations_month_state.groupBy("Month").pivot("UF").sum("Violations_Count")

# Show the resulting DataFrame
pivot_violations_month_state.show()

+-----+----+----+----+----+-----+-----+-----+-----+-----+----+-----+-----+-----+----+----+-----+-----+-----+-----+----+----+----+-----+-----+----+-----+----+
|Month|  AC|  AL|  AM|  AP|   BA|   CE|   DF|   ES|   GO|  MA|   MG|   MS|   MT|  PA|  PB|   PE|   PI|   PR|   RJ|  RN|  RO|  RR|   RS|   SC|  SE|   SP|  TO|
+-----+----+----+----+----+-----+-----+-----+-----+-----+----+-----+-----+-----+----+----+-----+-----+-----+-----+----+----+----+-----+-----+----+-----+----+
|   12| 992|3353|1881|2441|36934| 7253| 5360| 6001|18423|7123|39532|32609|17395|6727|5533|10972| 5969|19881|25031|7346|6909|1090|30135|16466|2932|39002|3575|
|    1|1935|4145|1960|1065|29613|11819| 3789| 9814|12965|6152|25886|19957|15282|4518|8921| 9599| 6897|28441|47616|5390|7160| 931|21648| 9393|3347|35375|3320|
|    6|1199|2133|1780|1048|33048| 7611| 7566|15296|18386|4014|39351|15812|14364|5398|5165| 9545| 5515|12865|47713|5028|7654| 610|15623|14244|1053|29731|3272|
|    3|1626|3892|1451|1136|38322|12586| 7749| 9687|2

In [458]:
# Import pandas lib
import pandas as pd

# Converting Spark Dataframe to Pandas Dataframe
df_pivot_violations_month_state = pivot_violations_month_state.toPandas()

In [459]:
df_pivot_violations_month_state

,Month,AC,AL,AM,AP,BA,CE,DF,ES,GO,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
0,12,992,3353,1881,2441,36934,7253,5360,6001,18423,...,19881,25031,7346,6909,1090,30135,16466,2932,39002,3575
1,1,1935,4145,1960,1065,29613,11819,3789,9814,12965,...,28441,47616,5390,7160,931,21648,9393,3347,35375,3320
2,6,1199,2133,1780,1048,33048,7611,7566,15296,18386,...,12865,47713,5028,7654,610,15623,14244,1053,29731,3272
3,3,1626,3892,1451,1136,38322,12586,7749,9687,26626,...,25538,45218,7639,9425,1255,23029,14372,3806,34265,3950
4,5,1056,2950,1658,1599,28583,11562,9927,14337,22472,...,18577,41165,6086,8190,703,19892,17870,3193,21472,3518
5,9,1082,4449,1801,1626,34947,10264,3117,16495,17573,...,15283,20197,7185,9287,1013,19967,16094,3539,30824,2282
6,4,1178,4175,2049,1670,42969,15011,13708,12860,25979,...,21734,48050,6544,8750,1523,23871,17591,3901,22893,3168
7,8,1109,2560,1520,1177,34006,10619,3562,15743,17892,...,16044,22501,6374,9161,907,18305,14365,2660,31345,2702
8,7,1491,1610,1450,876,32852,6897,8809,14789,18359,...,16983,36345,5192,6754,538,14714,12415,2429,31650,2698
9,10,1339,3767,1361,1827,41016,11813,3465,16637,16130,...,17804,24220,7429,7893,921,21528,17167,3874,35017,3019


In [460]:
px.imshow(
    df_pivot_violations_month_state,
    title=' Heatmap | Monthly Traffic Violations by State 2022')

Tipos Multas

In [461]:
# Compute the cumulative percentage
violation_type = TV22.select('Enquadramento_da_Infração') \
    .groupBy('Enquadramento_da_Infração') \
    .count() \
    .withColumn('cumulative_count', sum('count').over(Window.orderBy(col("count").desc()))) \
    .withColumn('cumulative_percentage', col('cumulative_count') / sum('count').over(Window.partitionBy()) * 100)

# Show the resulting DataFrame
violation_type.show(5)

+-------------------------+-------+----------------+---------------------+
|Enquadramento_da_Infração|  count|cumulative_count|cumulative_percentage|
+-------------------------+-------+----------------+---------------------+
|                    218 I|1093345|         1093345|   26.437889476131556|
|                      167| 228723|         1322068|   31.968580515692935|
|                    203 V| 206200|         1528268|    36.95464878323733|
|                  230 * V| 204476|         1732744|      41.899029457701|
|                   218 II| 182358|         1915102|   46.308580559218264|
+-------------------------+-------+----------------+---------------------+
only showing top 5 rows



In [462]:
# Loop no site do Detran e buscar os dados
Url = 'https://www.detran.mg.gov.br/infracoes/consultar-tipos-infracoes/index/index/index/index/index/index/index/index/index/index/index/index/index/lista-de-infracoes?artigo=&descricao=&page='

# Tabela vazia
df_violation_ticket_value = pd.DataFrame()

for Loop in range(1, 24):

    # Construindo a URL
    Link = f'{Url}{Loop}'

    # Lendo os dados da Web
    web_data = pd.read_html(Link)[0]

    # Consolidar
    df_violation_ticket_value = pd.concat([df_violation_ticket_value, web_data])

In [463]:
df_violation_ticket_value.head()

,Código,Desd.,Infração,Artigo,Infrator,Pts,Valor
0,5622,4,PARAR NOS CANTEIROS CENTRAIS OU DIVISORES DE P...,182 * VI,CON,3,8838
1,5622,5,PARAR NAS MARCAS DE CANALIZACAO,182 * VI,CON,3,8838
2,5630,0,PARAR NA AREA DE CRUZAMENTO DE VIAS,182 * VII,CON,4,13016
3,5649,1,PARAR NOS VIADUTOS,182 * VIII,CON,4,13016
4,5649,2,PARAR NAS PONTES,182 * VIII,CON,4,13016


In [464]:
df_violation_ticket_value['Valor'] = pd.to_numeric(df_violation_ticket_value['Valor'], errors='coerce')
df_violation_ticket_value['Valor'] = df_violation_ticket_value['Valor'] / 100

In [465]:
df_violation_ticket_value.head()

,Código,Desd.,Infração,Artigo,Infrator,Pts,Valor
0,5622,4,PARAR NOS CANTEIROS CENTRAIS OU DIVISORES DE P...,182 * VI,CON,3,88.38
1,5622,5,PARAR NAS MARCAS DE CANALIZACAO,182 * VI,CON,3,88.38
2,5630,0,PARAR NA AREA DE CRUZAMENTO DE VIAS,182 * VII,CON,4,130.16
3,5649,1,PARAR NOS VIADUTOS,182 * VIII,CON,4,130.16
4,5649,2,PARAR NAS PONTES,182 * VIII,CON,4,130.16


In [466]:
df_violation_ticket_value.rename( columns={'Código':'Código_da_Infração'}, inplace=True )

In [467]:
# Ajustando os dados de multas
# Codigos has more thna 1 type on infraction, when we try to merge all tables the repeted data will be duplicate
# Grouping by the violation code and getting the mean value will prevent this to happen
ticket_value = df_violation_ticket_value.groupby( by=['Código_da_Infração'] ).mean()['Valor'].reset_index()

codigo de infração é diferente nas tabelas, o ultimo digito não é obrigatorio então pdoemos arrumar isso no nosso dataframe de spark

In [468]:
TV22.show(5)

+---------------+---------------------------+----------------------+------------------+---------------+-----------+-----------+-----------+---------+-----------------------------+--------+-------------------------+-----------------------+----------------------+------------------------+------------------+----------------------------+-------------------------+---------------------------+---------------------+----------------+-------------+-------------------+------------------+-------------+
| Número_do_Auto|Data_da_Infração_DD/MM/AAAA|Indicador_de_Abordagem|Assinatura_do_Auto|Sentido_Trafego|UF_Infração|BR_Infração|Km_Infração|Município|Indicador_Veiculo_Estrangeiro|UF_Placa|Descrição_Especie_Veículo|Descrição_Marca_Veículo|Descrição_Tipo_Veículo|Descrição_Modelo_Veiculo|Código_da_Infração|Descrição_Abreviada_Infração|Enquadramento_da_Infração|Início_Vigência_da_Infração|Fim_Vigência_Infração|Medição_Infração|Hora_Infração|Medição_Considerada|Excesso_Verificado|Qtd_Infrações|
+---------

In [469]:
ticket_value.head()

,Código_da_Infração,Valor
0,5002,0.00
1,5010,880.41
2,5029,880.41
3,5037,586.94
4,5045,293.47


In [470]:
TV22 = TV22.withColumn('Código_da_Infração', col('Código_da_Infração').substr(1, 4))

In [471]:
TV22 = TV22.withColumn("Código_da_Infração", col("Código_da_Infração").cast("int"))

In [472]:
TV22.printSchema()

root
 |-- Número_do_Auto: string (nullable = true)
 |-- Data_da_Infração_DD/MM/AAAA: date (nullable = true)
 |-- Indicador_de_Abordagem: string (nullable = true)
 |-- Assinatura_do_Auto: string (nullable = true)
 |-- Sentido_Trafego: string (nullable = true)
 |-- UF_Infração: string (nullable = true)
 |-- BR_Infração: string (nullable = true)
 |-- Km_Infração: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Indicador_Veiculo_Estrangeiro: string (nullable = true)
 |-- UF_Placa: string (nullable = true)
 |-- Descrição_Especie_Veículo: string (nullable = true)
 |-- Descrição_Marca_Veículo: string (nullable = true)
 |-- Descrição_Tipo_Veículo: string (nullable = true)
 |-- Descrição_Modelo_Veiculo: string (nullable = true)
 |-- Código_da_Infração: integer (nullable = true)
 |-- Descrição_Abreviada_Infração: string (nullable = true)
 |-- Enquadramento_da_Infração: string (nullable = true)
 |-- Início_Vigência_da_Infração: string (nullable = true)
 |-- Fim_Vigência_Inf

In [473]:
violation_ticket_value = spark.createDataFrame(ticket_value)

In [498]:
violation_ticket_value = violation_ticket_value.withColumn("Código_da_Infração", col("Código_da_Infração").cast("int"))
violation_ticket_value = violation_ticket_value.withColumn("Valor", col("Valor").cast("float"))
violation_ticket_value = violation_ticket_value.na.fill(0, subset=["Valor"])

In [499]:
violation_ticket_value.printSchema()

root
 |-- Código_da_Infração: integer (nullable = true)
 |-- Valor: float (nullable = false)



In [500]:
TV22 = TV22.join(violation_ticket_value, on='Código_da_Infração', how='left')

In [501]:
TV22.printSchema()

root
 |-- Código_da_Infração: integer (nullable = true)
 |-- Número_do_Auto: string (nullable = true)
 |-- Data_da_Infração_DD/MM/AAAA: date (nullable = true)
 |-- Indicador_de_Abordagem: string (nullable = true)
 |-- Assinatura_do_Auto: string (nullable = true)
 |-- Sentido_Trafego: string (nullable = true)
 |-- UF_Infração: string (nullable = true)
 |-- BR_Infração: string (nullable = true)
 |-- Km_Infração: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Indicador_Veiculo_Estrangeiro: string (nullable = true)
 |-- UF_Placa: string (nullable = true)
 |-- Descrição_Especie_Veículo: string (nullable = true)
 |-- Descrição_Marca_Veículo: string (nullable = true)
 |-- Descrição_Tipo_Veículo: string (nullable = true)
 |-- Descrição_Modelo_Veiculo: string (nullable = true)
 |-- Descrição_Abreviada_Infração: string (nullable = true)
 |-- Enquadramento_da_Infração: string (nullable = true)
 |-- Início_Vigência_da_Infração: string (nullable = true)
 |-- Fim_Vigência_Inf

In [513]:
uf_tickets_value = TV22.groupBy('UF_Infração').agg(
    count('Valor').alias('Violations_Count'),
    sum('Valor').alias('Total_ticket_value')
)

In [514]:
uf_tickets_value.show(5)

+-----------+----------------+--------------------+
|UF_Infração|Violations_Count|  Total_ticket_value|
+-----------+----------------+--------------------+
|         SC|          178040|  3.88509024023819E7|
|         RO|           93105|2.0328607025260925E7|
|         PI|           85095| 2.318876397544861E7|
|         AM|           20102|   4812790.681762695|
|         RR|           12033|   3387469.271133423|
+-----------+----------------+--------------------+
only showing top 5 rows



In [515]:
# Import pandas lib
import pandas as pd

# Converting Spark Dataframe to Pandas Dataframe
df_uf_tickets_value = uf_tickets_value.toPandas()

In [517]:
px.scatter(
    df_uf_tickets_value,
    x='Violations_Count',
    y='Total_ticket_value',
    color='UF_Infração',
    size='Violations_Count',
    log_x=True,
    size_max=60,
    title='Bubble PLOT' )

In [519]:
fig = px.scatter(df_uf_tickets_value, x='Violations_Count', y='Total_ticket_value', color='UF_Infração', title='Scatter PLOT')
fig.update_traces(marker=dict(size=12) )